# scaling mphi seg 3 class

In [4]:
import napari
from skimage import io
import btrack
import os
from tqdm.auto import tqdm
import numpy as np
import btrack
from glob import glob
from homuncu_loc.dataio import find_files_with_basename, ID_extractor

#### Define image fn 

In [3]:
# print gpu information
!nvcc --version
!nvidia-smi

# load cellpose
from cellpose import core, utils, models, metrics
import matplotlib.pyplot as plt 
from skimage.morphology import remove_small_objects

# check to see if GPU can be used
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

# define segmentation model parameters
model = models.Cellpose(gpu=use_GPU, 
                        model_type='cyto') # cytoplasmic segmentation 
channels = [0,0] # this means using a grayscale image for both nuclei and cyto channels (even if not using nuclei, still have to say its same colour [greyscale = 0])

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Wed Sep  6 13:16:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:65:00.0  On |                  Off |
| 30%   49C    P2    80W / 300W |   2450MiB / 49140MiB |      4%      Default |
|                               |                      |                  N/A |
+

# define a general segmentation framework for macroph and edit params for other cell

In [5]:
from tqdm.auto import tqdm
from cellpose import models
from skimage.morphology import remove_small_objects
import numpy as np
import btrack

def segment_channel(segmentation_channel, diameter=60, flow_threshold=0.99, channels=[0,0], cellprob_threshold=-2, min_mask_size=500, return_diams=False, start_point=None, end_point=None):
    """
    Segments a specified channel of an image using Cellpose and optionally returns the average diameter of segments.

    Args:
    - segmentation_channel (ndarray): 3D array of the target channel where the 3rd dimension represents the Z-stack.
    - diameter (int, optional): Average cell diameter for Cellpose. Default is 60.
    - flow_threshold (float, optional): Threshold for flow in Cellpose. Default is 0.99.
    - channels (list, optional): Channel input for Cellpose. Default is [0,0].
    - cellprob_threshold (float, optional): Cell probability threshold for Cellpose. Default is -2.
    - min_mask_size (int, optional): Minimum size of objects to keep in the segmented mask. Default is 500.
    - return_diams (bool, optional): If set to True, the function will also return the average diameter of segments.
    - start_point (int, optional): Starting point for segmentation frames. Default is None.
    - end_point (int, optional): Ending point for segmentation frames. Default is None.

    Returns:
    - ndarray: 3D mask array with the same shape as the input, where segmented regions are labeled.
    - float (optional): Average diameter of segments if return_diams is True.
    """

    if start_point is None or end_point is None:
        # Calculate intensities for each frame in Z-stack
        intensities = [np.sum(frame) for frame in tqdm(segmentation_channel, desc='finding where segmentation channel is expressed in Z')]

        # Estimate the background level
        background_level = np.min(intensities)
        threshold = background_level + ((np.max(intensities)-np.min(intensities))*0.50)

        if start_point is None:
            # Find the points where the curve starts to deviate from the background
            start_point = np.argmax(intensities > threshold)

        if end_point is None:
            end_point = len(intensities) - np.argmax(intensities[::-1] > threshold) - 1

    # Initialize output mask array
    mask_array = np.zeros_like(segmentation_channel)

    # Initialize the model
    model = models.Cellpose(model_type='cyto')
    
    # List to store diameters if return_diams is True
    diams = []

    # Segment over selected frames
    for n, frame in tqdm(enumerate(segmentation_channel[start_point:end_point]), total=end_point-start_point, desc='segmenting over select frames'):
        n += start_point
        masks, _, _, diam = model.eval(frame, diameter=diameter, flow_threshold=flow_threshold, channels=channels, cellprob_threshold=cellprob_threshold)
        masks = remove_small_objects(masks, min_size=min_mask_size)
        mask_array[n] = masks
        diams.append(diam)

    if return_diams:
        return mask_array, diams
    else:
        return mask_array


In [6]:
props = ('axis_major_length', # first define some cell properties 
         'axis_minor_length', # this helps improve the tracking
         'eccentricity',      # by comparing similarities between slices/frames
         'area', 
         'orientation',
         'mean_intensity', 
        )

In [7]:
basedir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images'

In [8]:
image_basenames = [
    "20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1298",
    "20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1299",
    "20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1300",
    "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1306",
    "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1307",
    "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1308",
    "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1309",
    "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1310",
    "20230705_40X_23-01-005A3_Multichannel Z-Stack_20230705_1311",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1315",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1316",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1317",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1318",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1319",
    "20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1320",
    "20230707_40X_23-01-005A3_Multichannel Z-Stack_20230707_1324",
    "20230707_40X_23-01-005A3_Multichannel Z-Stack_20230707_1325",
    "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1340",
    "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1341",
    "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1342",
    "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1343",
    "20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1344",
    "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1361",
    "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1362",
    "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363",
    "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364",
    "20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365",
    "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1463",
    "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1464",
    "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1465",
    "20230801_20X_23-03-001B4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230801_1467",
    "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1482",
    "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1483",
    "20230801_20X_23-03-001B2_DAPI_ZO-1_Ki67_Mtb_Multichannel Z-Stack_20230801_1484",
    "20230802_20X_23-03-001A4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1500",
    "20230802_20X_23-03-001A4_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1501",
    "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1510",
    "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1511",
    "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1512",
    "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1513",
    "20230802_20X_23-03-001A1_DAPI_ZO-1_CD16_Mtb_Multichannel Z-Stack_20230802_1514"

]


In [9]:
image_fns = [os.path.join(basedir, fn+'.tif') for fn in image_basenames]

In [10]:
from macrohet.notify import send_sms

In [25]:
%%time 
image_fn = os.path.join(basedir, '20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1299_test.tif')
print(image_fn)
image = io.imread(image_fn)
send_sms(f'image {ID_extractor(image_fn)} loaded')

/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1299_test.tif
CPU times: user 2.96 s, sys: 7.86 s, total: 10.8 s
Wall time: 2min 14s


In [95]:
%%time 
image_fn = image_fns[27]
print(image_fn)
image = io.imread(image_fn)
send_sms(f'image {ID_extractor(image_fn)} loaded')

/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364.tif


TwilioRestException: HTTP 401 error: Unable to create record: Authenticate

In [96]:
viewer = napari.Viewer(title = f'{ID_extractor(image_fn)}')

viewer.add_image(image, channel_axis=-1)

[<Image layer 'Image' at 0x7f1af8f94160>,
 <Image layer 'Image [1]' at 0x7f1af8da0a60>,
 <Image layer 'Image [2]' at 0x7f221ae0a0d0>,
 <Image layer 'Image [3]' at 0x7f221ae36d30>]

## Load corresponding sc data

In [97]:
glob(image_fn.replace('images', 'sc_analyses').replace('.tif', '*.h5'))

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364_z_tracks_masks_d100.h5',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364_z_tracks_masks.h5']

In [98]:
# Iterate over the file paths
for file_path in glob(image_fn.replace('images', 'sc_analyses').replace('.tif', '*.h5')):
    # Get the file size in bytes
    file_size_bytes = os.path.getsize(file_path)

    # Convert the file size to MB
    file_size_mb = file_size_bytes / (1024 * 1024)  # 1 MB = 1024 KB, 1 KB = 1024 bytes

    # Extract the last substring of the file basename from "1361" onwards
    basename = os.path.basename(file_path)
    last_substring = basename.split(ID_extractor(image_fn))[-1]

    # Print the file size in MB and the last substring
    print(f"File: {ID_extractor(image_fn)}{last_substring}, Size (MB): {file_size_mb:.2f}")

File: 1364_z_tracks_masks_d100.h5, Size (MB): 5.77
File: 1364_z_tracks_masks.h5, Size (MB): 6.34


In [92]:
i = glob(image_fn.replace('images', 'sc_analyses').replace('.tif', '*.h5'))[2]
with btrack.io.HDF5FileHandler(i, 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation
    tracks = [t for t in reader.tracks if len(t) >= 3]

napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks, name = i.split(ID_extractor(i))[-2])

[INFO][2023/09/06 03:12:34 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363_z_tracks_masks_d100.h5...
[INFO][2023/09/06 03:12:37 pm] Loading segmentation (81, 2304, 2304)
[INFO][2023/09/06 03:12:37 pm] Loading tracks/obj_type_1
[INFO][2023/09/06 03:12:37 pm] Loading LBEP/obj_type_1
[INFO][2023/09/06 03:12:37 pm] Loading objects/obj_type_1 (4959, 5) (4959 filtered: None)
[INFO][2023/09/06 03:12:37 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363_z_tracks_masks_d100.h5


<Tracks layer '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363_z_tracks_masks_' at 0x7f1bc67e9610>

In [48]:
sc_fn = image_fn.replace('images', 'sc_analyses').replace('.tif', '_sc_data.h5')

In [129]:
with btrack.io.HDF5FileHandler(sc_fn, 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation
    tracks = [t for t in reader.tracks if len(t) >= 3]

napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

[INFO][2023/09/06 12:57:45 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1319_z_tracks_masks_d100.h5...
[INFO][2023/09/06 12:57:58 pm] Loading segmentation (317, 2304, 2304)
[INFO][2023/09/06 12:57:58 pm] Loading tracks/obj_type_1
[INFO][2023/09/06 12:57:58 pm] Loading LBEP/obj_type_1
[INFO][2023/09/06 12:57:58 pm] Loading objects/obj_type_1 (21159, 5) (21159 filtered: None)
[INFO][2023/09/06 12:58:01 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1319_z_tracks_masks_d100.h5


<Tracks layer 'napari_tracks [1]' at 0x7f5931165ac0>

In [130]:
print()